In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
import numpy as np
import cv2
import os
import h5py
from imutils import face_utils
from keras.models import load_model
import sys
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras.layers import Dense, Activation, Flatten
from keras.utils import to_categorical
from keras import backend as K
from sklearn.model_selection import train_test_split
from Model_face_recogination import model
from keras import callbacks
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import csv
import datetime


In [ ]:
path = r'Path to your dataset'

In [ ]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");


def downsample_image(img):
    img = Image.fromarray(img.astype('uint8'), 'L')
    img = img.resize((224, 224), Image.LANCZOS)
    return np.array(img)


# function to get the images and label data
def getImagesAndLabels(path):
    path = r'Path to your dataset'
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]

    class_id_map = {}
    next_id = 1
    faceSamples = []
    ids = []

    for imagePath in imagePaths:
        # If there is an error saving any jpegs
        try:
            PIL_img = Image.open(imagePath).convert('L')  # convert it to grayscale
        except:
            continue
        img_numpy = np.array(PIL_img, 'uint8')

        # Extract class name from image path
        file_name = os.path.split(imagePath)[-1]
        class_name = file_name.split(".")[1]  # Extract the class name after the dot

        # Assign a unique ID to the class if it's not already assigned
        if class_name not in class_id_map:
            class_id_map[class_name] = next_id
            next_id += 1  # Increment the ID counter

        id = class_id_map[class_name]
        faceSamples.append(img_numpy)
        ids.append(id)

    return faceSamples, ids, class_id_map

faces, ids, class_id_map = getImagesAndLabels(path)

In [ ]:
K.clear_session()
n_faces = len(set(ids))
model = model((224, 224, 3), n_faces)
# faces = np.asarray(faces)
faces = np.array([downsample_image(ab) for ab in faces])
ids = np.asarray(ids)
faces = faces[:, :, :, np.newaxis]
print("Shape of Data: " + str(faces.shape))
print("Number of unique faces : " + str(n_faces))


In [ ]:
ids = to_categorical(ids)


print(len(ids))

faces = faces.astype('float32')
faces /= 255.

x_train, x_test, y_train, y_test = train_test_split(
    faces, ids, test_size=0.1, random_state=0)
x_train_rgb = np.repeat(x_train, 3, axis=-1)  # This will change the shape to (num_samples, 224, 224, 3) RGB images
x_test_rgb = np.repeat(x_test, 3, axis=-1)



In [ ]:
checkpoint = callbacks.ModelCheckpoint(r'Path to save model ', verbose=1, monitor='val_loss', save_best_only=True)

checkpoint1 = callbacks.ModelCheckpoint(r'Path to save model.h5 file',save_best_only=True, save_weights_only=True, verbose=1)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6, verbose=1)

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

callbacks = [checkpoint1, checkpoint, reduce_lr,early_stop]




In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True)

In [ ]:
history = model.fit(datagen.flow(x_train_rgb, y_train, batch_size=32),
          steps_per_epoch=40,
          epochs=120,
          validation_data=(x_test_rgb, y_test),
          shuffle=True, callbacks=callbacks)

In [ ]:
# Convert integer IDs to string IDs
reverse_class_id_map = {v: k for k, v in class_id_map.items()}
ids_list = ids.argmax(axis=1)
ids_str = [reverse_class_id_map[i] for i in ids_list]

print("Classes:", class_id_map)



# Print the number of faces trained and end program
print("\n [INFO] " + str(n_faces) + " faces trained. Exiting Program")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from keras.models import load_model

# Load the trained model
model = load_model(r'Path to load your model')
loss, accuracy = model.evaluate(
    x=x_test_rgb,
    y=y_test,
    batch_size=32)

In [ ]:
# Print the test loss and accuracy
print("Test Loss:", loss)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
import cv2
import numpy as np
import os
from keras.models import load_model
import csv
import datetime

# Load the trained model
model = load_model(r'Path to load your model')

# Load the face detection classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Directory containing images
image_folder = r'Path to test images'

# Initialize CSV File for logging attendance
csv_file = r'path to your csv file to take attendance'
fieldnames = ['ID', 'Name', 'Date', 'Time', 'File']
with open(csv_file, 'a', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    if file.tell() == 0:  # Write header only if file is empty
        writer.writeheader()

def log_attendance(person_id, person_name, image_file):
    with open(csv_file, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writerow({
            'ID': person_id,
            'Name': person_name,
            'Date': datetime.date.today().isoformat(),
            'Time': datetime.datetime.now().strftime("%H:%M:%S"),
            'File': os.path.basename(image_file)
        })

# List all image files in the directory
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Iterate over each image file
for image_file in image_files:
    # Read the image
    frame = cv2.imread(image_file)

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # For each detected face, classify and print the prediction
    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face_region = gray[y:y+h, x:x+w]

        face_region_preprocessed = downsample_image(face_region)
        face_region_preprocessed = np.stack((face_region_preprocessed,)*3, axis=-1)  # Convert to 3 channels
        face_region_preprocessed = face_region_preprocessed.reshape(1, 224, 224, 3) / 255.0  # Add batch dimension and normalize

        # Predict the identity using your model
        predictions = model.predict(face_region_preprocessed)
        predicted_id = np.argmax(predictions)
        predicted_identity = reverse_class_id_map[predicted_id] if predicted_id in reverse_class_id_map else "Unknown"

        # Log the attendance in the CSV file
        log_attendance(predicted_id, predicted_identity, image_file)

        # Display the predicted identity and ID on the frame
        display_text = f"ID: {predicted_id}, Name: {predicted_identity}"
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, display_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        print(display_text)

    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Break the loop with the 'q' key
    if cv2.waitKey(200) & 0xFF == ord('q'):
        break

# Close all OpenCV windows
cv2.destroyAllWindows()


In [ ]:
# Load your trained model
model = load_model(r'Path to load your model')

# Initialize the video capture object
cap = cv2.VideoCapture(0)
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# CSV file setup
csv_file = r'path to your csv file to take attendance'
fieldnames = ['ID', 'Name', 'Date', 'Time']
with open(csv_file, 'a', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    if file.tell() == 0:  # Write header only if file is empty
        writer.writeheader()

def log_attendance(person_id, person_name):
    with open(csv_file, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writerow({
            'ID': person_id,
            'Name': person_name,
            'Date': datetime.date.today().isoformat(),
            'Time': datetime.datetime.now().strftime("%H:%M:%S")
        })

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the captured frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the face region
        face_region = gray[y:y+h, x:x+w]

        # Preprocess the face region (resize, reshape, etc.) to match your model's input
        face_region_preprocessed = downsample_image(face_region)
        face_region_preprocessed = np.stack((face_region_preprocessed,)*3, axis=-1)  # Convert to 3 channels
        face_region_preprocessed = face_region_preprocessed.reshape(1, 224, 224, 3) / 255.0  # Add batch dimension and normalize

        # Predict the identity using your model
        predictions = model.predict(face_region_preprocessed)
        predicted_id  = np.argmax(predictions)
        predicted_identity = reverse_class_id_map[predicted_id] if predicted_id in reverse_class_id_map else "Unknown"

        # Log the attendance in the CSV file
        log_attendance(predicted_id, predicted_identity)

        # Display the predicted identity and ID on the frame
        display_text = f"ID: {predicted_id}, Name: {predicted_identity}"
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, display_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Break the loop with the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
